In [ ]:
from transformers import MambaConfig, MambaForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-130m-hf")
model = MambaForCausalLM.from_pretrained("state-spaces/mamba-130m-hf").to("cuda").half()
input_ids = tokenizer("Hey how are you doing?", return_tensors= "pt")["input_ids"].to("cuda")

out = model.generate(input_ids, max_new_tokens=10)
print(tokenizer.batch_decode(out))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/4.79k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/895 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/517M [00:00<?, ?B/s]

The fast path is not available because one of `(selective_state_update, selective_scan_fn, causal_conv1d_fn, causal_conv1d_update, mamba_inner_fn)` is None. Falling back to the sequential implementation of Mamba, as use_mambapy is set to False. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d. For the mamba.py backend, follow https://github.com/alxndrTL/mamba.py.


generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


["Hey how are you doing?\n\nI'm so glad you're here."]


In [ ]:
len(tokenizer.vocab)

50277

In [ ]:
model2 = MambaForCausalLM.from_pretrained("state-spaces/mamba-130m-hf").to("cuda").half()

In [ ]:
model(input_ids)

MambaCausalLMOutput(loss=None, logits=tensor([[[-55.6875, -69.6875, -49.9062,  ..., -69.9375, -69.6250, -69.7500],
         [-43.6562, -59.9375, -40.0938,  ..., -60.2812, -59.9062, -60.0000],
         [-34.1875, -52.2188, -34.6562,  ..., -52.2500, -52.2500, -52.1562],
         [-51.9062, -68.5625, -48.3750,  ..., -68.8750, -68.6250, -68.5625],
         [-55.5000, -69.5625, -50.5000,  ..., -69.7500, -69.5625, -69.5625],
         [ -6.7539, -24.7812,  -6.5234,  ..., -24.8750, -24.9219, -24.7656]]],
       device='cuda:0', grad_fn=<ToCopyBackward0>), cache_params=<transformers.cache_utils.MambaCache object at 0x78e63dcef010>, hidden_states=None)

In [ ]:
out

tensor([[8262,  849,  403,  368, 2509,   32,  187,  187,   42, 1353,  594, 9995,
          368, 1472, 1060,   15]])

In [ ]:
model

MambaForCausalLM(
  (backbone): MambaModel(
    (embeddings): Embedding(50280, 768)
    (layers): ModuleList(
      (0-23): 24 x MambaBlock(
        (norm): MambaRMSNorm(768, eps=1e-05)
        (mixer): MambaMixer(
          (conv1d): Conv1d(1536, 1536, kernel_size=(4,), stride=(1,), padding=(3,), groups=1536)
          (act): SiLU()
          (in_proj): Linear(in_features=768, out_features=3072, bias=False)
          (x_proj): Linear(in_features=1536, out_features=80, bias=False)
          (dt_proj): Linear(in_features=48, out_features=1536, bias=True)
          (out_proj): Linear(in_features=1536, out_features=768, bias=False)
        )
      )
    )
    (norm_f): MambaRMSNorm(768, eps=1e-05)
  )
  (lm_head): Linear(in_features=768, out_features=50280, bias=False)
)

In [ ]:
# prompt: create tensor 5*768

import torch

# Create a tensor of size 5x768 with random values
tensor = torch.randn(2,15, 768).to("cuda")

tensor

tensor([[[ 0.6130,  0.1739, -0.0190,  ...,  1.3567,  0.5717,  0.9927],
         [-0.6612, -1.9666, -0.6580,  ..., -0.6200,  1.0751,  0.3166],
         [-0.1575, -1.4532,  2.0137,  ...,  0.8022,  0.9559,  0.9234],
         ...,
         [-1.0359, -2.1492,  2.1647,  ..., -0.1188,  0.4520,  0.2344],
         [-1.7552, -1.8183,  0.2846,  ...,  0.3913, -1.8381, -0.1768],
         [-0.2294,  1.5713,  0.9124,  ..., -0.6739,  0.8656,  0.7350]],

        [[-0.1705, -0.9806,  2.2900,  ..., -2.7920,  0.4469, -0.9085],
         [-2.3693,  0.0245, -1.8061,  ..., -2.3947, -0.5658, -0.0159],
         [ 0.3280,  0.4251, -0.8758,  ..., -0.5298, -1.4926,  1.8532],
         ...,
         [-1.4351,  0.6479,  1.5455,  ..., -0.1380,  0.7814,  0.6462],
         [-0.1176,  0.0989,  0.1185,  ..., -0.4915, -1.5701,  0.2679],
         [-0.2898, -1.9689, -0.5886,  ..., -0.3171, -0.6597,  0.2550]]],
       device='cuda:0')

In [ ]:
res = model.backbone.layers[0].forward(tensor)

In [ ]:
res.shape

torch.Size([2, 15, 768])

# text tokenizer from mamba

In [ ]:
input_ids = tokenizer("Hey how are you doing?", return_tensors= "pt")["input_ids"]

input_ids

tensor([[8262,  849,  403,  368, 2509,   32]])

# audio encoder, wavlm

In [ ]:
# !pip install datasets

In [ ]:
inputs

{'input_values': tensor([[0.0024, 0.0021, 0.0020,  ..., 0.0004, 0.0006, 0.0010]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], dtype=torch.int32)}

In [ ]:
from transformers import AutoProcessor, WavLMModel
import torch
from datasets import load_dataset

dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation", trust_remote_code=True)
dataset = dataset.sort("id")
sampling_rate = dataset.features["audio"].sampling_rate

processor = AutoProcessor.from_pretrained("patrickvonplaten/wavlm-libri-clean-100h-base-plus")
wavlm_model = WavLMModel.from_pretrained("patrickvonplaten/wavlm-libri-clean-100h-base-plus")

# audio file is decoded on the fly
inputs = processor(dataset[0]["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")
with torch.no_grad():
    outputs = wavlm_model(inputs.input_values)

last_hidden_states = outputs.last_hidden_state
list(last_hidden_states.shape)

[1, 292, 768]

# audio + text model

In [ ]:
import torch.nn as nn

In [ ]:
test_text= "please do summarizatoin for the audio."

audio= dataset[0]["audio"]["array"]

def get_pred(audio):
    enc_out_text = tokenizer(test_text, return_tensors="pt")

    enc_out_text =model.backbone.embeddings(enc_out_text.input_ids.to("cuda"))

    enc_out_audio = processor(audio, return_tensors="pt")

    enc_out_audio =wavlm_model.feature_extractor(enc_out_audio.input_values)

    audio_adaptor = nn.Linear(512, 768)

    enc_out_audio = audio_adaptor(enc_out_audio.reshape(1, 292, 512)).to('cuda')


    enc_concat = torch.cat([enc_out_text, enc_out_audio], dim=1) # make it trainable



    res = model.backbone.layers[0].forward(enc_concat)


    pred=model.lm_head.forward(res.reshape(1, 302, 768).type(model.lm_head.weight.dtype))
    return pred

In [ ]:
target_text= "please do summarizatoin for the audio. Result is: the bird is flying in the sky."

target_tensor = tokenizer(target_text, return_tensors="pt")["input_ids"]

In [ ]:
import torch.nn.functional as F


# Assuming target_tensor is 1D
current_length = target_tensor.shape[1]
padding_length = 302 - current_length

# Pad the tensor with zeros at the end
padded_target_tensor = F.pad(target_tensor, (0, padding_length), 'constant', 0)

In [ ]:
padded_target_tensor.shape

torch.Size([1, 302])

In [ ]:
import torch
import torch.nn as nn

def train(audio, text):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(audio_adaptor.parameters())

    loss = criterion(pred.reshape(1, 50280, 302), padded_target_tensor.to('cuda'))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# falcon mamba

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/falcon-mamba-7b")
model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/falcon-mamba-7b").half().to("cuda")

input_text = "Question: How many hours in one day? Answer: "
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], min_new_tokens= 100))


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Question: How many hours in one day? Answer: 24 hours.
Question: How many


In [ ]:
input_text = "please create one story about tiger in 100 words in chinese."
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids, min_new_tokens= 200)
print(tokenizer.decode(outputs[0]))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1399: UserWarning: Unfeasible length constraints: `min_length` (213) is larger than the maximum possible length (20). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length. Note that `max_length` is set to 20, its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1407: UserWarning: Unfeasible length constraints: `min_new_tokens` (200), when added to the prompt length (13), is larger than the maximum possible length (20). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length. Note that `max_length` is set to 20, its default value.
  warnings.warn(


please create one story about tiger in 100 words in chinese.
The tiger is a big and


In [ ]:
model.backbone.layers[0]

FalconMambaBlock(
  (norm): FalconMambaRMSNorm(4096, eps=1e-05)
  (mixer): FalconMambaMixer(
    (conv1d): Conv1d(8192, 8192, kernel_size=(4,), stride=(1,), padding=(3,), groups=8192)
    (act): SiLU()
    (in_proj): Linear(in_features=4096, out_features=16384, bias=False)
    (x_proj): Linear(in_features=8192, out_features=288, bias=False)
    (dt_proj): Linear(in_features=256, out_features=8192, bias=True)
    (out_proj): Linear(in_features=8192, out_features=4096, bias=False)
  )
)

In [ ]:
!pip install --upgrade transformers

from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig

# Load the configuration file directly to identify the correct model type
config = AutoConfig.from_pretrained("tiiuae/falcon-mamba-7b")
print(f"Model type detected: {config.model_type}")

# Check if the model type is supported by AutoModelForCausalLM
# and if not, try to load with the correct class based on the model type
if config.model_type == "falcon_mamba":
    # If falcon_mamba is a Causal Language Model, this should work:
    model = AutoModelForCausalLM.from_pretrained("tiiuae/falcon-mamba-7b", trust_remote_code=True)
else:
    # If it's a different type, you might need to use a different class.
    # This is just an example, you'll need to find the right class for the specific model
    # from transformers import AutoModelForSeq2SeqLM
    # model = AutoModelForSeq2SeqLM.from_pretrained("tiiuae/falcon-mamba-7b", trust_remote_code=True)
    raise ValueError(f"Unsupported model type: {config.model_type}")


tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-mamba-7b", trust_remote_code=True)


input_text = "Question: How many hours in one day? Answer: "
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Model type detected: falcon_mamba
